In [1]:
import pandas as pd
import numpy as np
import os
import shinobi_behav
import glob
import os.path as op
from shinobi_behav import DATA_PATH, FIG_PATH
import nibabel as nib
import json
import matplotlib.pyplot as plt
import retro
from datetime import datetime
from itertools import product
import dataframe_image as dfi

def count_frames(bk2_path):
    movie = retro.Movie()
    frame_count = 0
    while movie.step():
        frame_count += 1
    return frame_count

def get_rep_info(bk2_path):
    json_fname = bk2_path.replace(".bk2", ".json")
    with open(json_fname) as f:
        sidecar = json.load(f)
        duration = sidecar["duration"]
        cleared = sidecar["cleared"]
        kills = sidecar["enemies killed"]
        healthlosses = sidecar["total health lost"]
    return duration, cleared, kills, healthlosses




In [2]:


def get_behav_df(replayfile_list):
    sub_list = []
    run_list = []
    ses_list = []
    duration_dict = []
    clear_dict = []
    kill_dict = []
    healthloss_dict = []

    data_dict = {"subject":[],
                "session":[],
                "run":[],
                "level":[],
                "N":[],
                "duration (h:m:s)":[],
                "cleared":[],
                "kills":[],
                "healthlosses":[]}
    for replayfile in replayfile_list:
        if op.isfile(replayfile.replace(".bk2", ".json")):
            duration, cleared, kills, healthlosses = get_rep_info(replayfile)
            data_dict["subject"].append(replayfile.split('/')[7])
            data_dict["session"].append(replayfile.split('/')[8])
            data_dict["run"].append(replayfile.split('/')[-1].split('_')[3].split('-')[1])
            data_dict["level"].append(replayfile.split('/')[-1].split('_')[-2].replace('-', ' '))
            data_dict["duration (h:m:s)"].append(float(duration))
            data_dict["cleared"].append(int(cleared))
            data_dict["kills"].append(int(kills))
            data_dict["healthlosses"].append(int(healthlosses))
            data_dict["N"].append(1)

    data_df = pd.DataFrame(data_dict)    
    #data_df['duration'] = pd.to_timedelta(data_df['duration'], unit='s').dt.components[['hours', 'minutes', 'seconds']].astype(str).agg(':'.join, axis=1)
    dataset_fname = op.join(DATA_PATH, "processed", "descriptive_scan_full.csv")
    data_df.to_csv(op.join(DATA_PATH, "processed", "descriptive_scan_full.csv"), index=False)
    return data_df



replayfile_list = sorted(glob.glob(op.join(DATA_PATH, "shinobi_released", "shinobi", "*", "*", "*", "*.bk2")))
data_df = get_behav_df(replayfile_list)

'1 day 07:49:43'

,N,cleared,duration (h:m:s)
subject,,,
sub-01,206.0,181.0,28981.416667
sub-02,194.0,153.0,30676.983333
sub-04,179.0,143.0,29559.100000
sub-06,101.0,80.0,25365.916667
Total,680.0,557.0,114583.416667


In [82]:
# Pivot table
#data_df = pd.read_csv("../tmp/shinobi_dataset_description_test.csv")

def make_descriptive_table(data_df):
    data_df.drop(["kills", "run", 'healthlosses'], axis=1, inplace=True)
    data_pivot = data_df.pivot_table(index='subject', columns='level', aggfunc=np.sum).swaplevel(axis=1).sort_index(axis=1)

    # Add total column
    total_df = data_df.groupby("subject").sum()
    total_df = total_df[["N", "cleared", "duration (h:m:s)"]]
    multiindex_tuples = [x for x in product(['Total'], total_df.columns)]
    total_df.columns = pd.MultiIndex.from_tuples(multiindex_tuples)
    data_pivot = pd.concat([data_pivot, total_df], axis=1)

    # Add total row
    data_pivot.loc['Total'] = data_pivot.sum()
    # Correct total total


    # Convert seconds to H:M:S
    def seconds_to_time(seconds):
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)
        # Return time string
        return f'{int(hours):02}:{int(minutes):02}:{int(seconds):02}'
    # Convert time
    def convert_seconds_to_time(df, col):
        df[col] = df[col].apply(seconds_to_time)
    # Apply conversion function to each sub-column
    for col in data_pivot.columns:
        if col[1] == 'duration (h:m:s)':
            convert_seconds_to_time(data_pivot, col)

    # Convert all to int
    for col in data_pivot.select_dtypes(include=['float64']):
        data_pivot[col] = data_pivot[col].astype(int)

    #data_pivot.to_csv(op.join(DATA_PATH, "processed", "descriptive_scan_pivot.csv"))
    return data_pivot

data_pivot = make_descriptive_table(data_df)
data_pivot

level   level 1                          level 4                           \
              N cleared duration (h:m:s)       N cleared duration (h:m:s)   
subject                                                                     
sub-01       89      84         03:57:26      64      44         01:59:50   
sub-02       81      76         03:41:53      58      30         02:11:49   
sub-04       70      67         03:26:50      56      29         02:06:00   
sub-06       46      45         03:29:24      42      23         02:44:22   
Total       286     272         14:35:34     220     126         09:02:02   

level   level 5                          Total                           
              N cleared duration (h:m:s)     N cleared duration (h:m:s)  
subject                                                                  
sub-01       53      53         02:05:44   206     181         08:03:01  
sub-02       55      47         02:37:34   194     153         08:31:16  
sub-04       53      47         02:39:47   179     143         08:12:39  
sub-06       13      12         00:48:59   101      80         07:02:45  
Total       174     159         08:12:06   680     557         31:49:43

In [80]:
dfi.export(data_pivot, op.join("../reports", 'repetitions_description_scan.png'))

In [33]:
replayfile_list

Total                         
            N duration (h:m:s) cleared
subject                               
sub-01    206     28981.416667     181
sub-02    194     30676.983333     153
sub-04    179     29559.100000     143
sub-06    101     25365.916667      80

## Training

In [4]:
replayfile_list

['/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-001/beh/sub-01_ses-001_task-shinobi_level-1_rep-001_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-002/beh/sub-01_ses-002_task-shinobi_level-1_rep-001_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-002/beh/sub-01_ses-002_task-shinobi_level-1_rep-002_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-001_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-002_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-003_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-004_gamedata.bk2',
 '/scratch/hyruuk/neuromod/shinobi2023/shinobi_training/sub-01

In [3]:
replayfile_list = sorted(glob.glob(op.join(DATA_PATH, "shinobi_training", "*", "*", "*", "*.bk2")))
data_df = get_behav_df(replayfile_list)

KeyError: 'duration'

In [ ]:
data_pivot = make_descriptive_table(data_df)
data_pivot

In [ ]:
# Number of usable repetitions

usable = np.sum(df["nrep_usable"])
total = np.sum(df["nrep_total"])
print(f"{usable}/{total}")

In [ ]:
# total number of repetitions : 
grouped = df[["sub", "nlvl1", "nlvl4", "nlvl5"]].groupby('sub').sum()
grouped.loc['Total'] = grouped.sum(axis=0)
grouped['Total'] = grouped.sum(axis=1)
grouped

In [ ]:
df_without_total = grouped.drop(columns='Total')
df_cumsum = df_without_total.cumsum(axis=1)

fig, ax = plt.subplots(figsize=(10, 6))

# Create a bar for each level, with the bottom at the cumulative sum of the previous levels
for i, col in enumerate(df_without_total.columns):
    ax.bar(df_without_total.index, df_without_total[col], bottom=(df_cumsum.iloc[:, i-1] if i>0 else 0), label=col)

plt.xlabel('Subject')
plt.ylabel('Count')
plt.title('Number of repetitions per level and per subject')
plt.legend()
plt.show()

In [ ]:

eventsfile_list = sorted(glob.glob(op.join(DATA_PATH, "shinobi_released", "shinobi", "*", "*", "*", "*_desc-annotated_events.tsv")))
#def build_dataset_description(dataset_fname):
sub_list = []
run_list = []
ses_list = []
fmrifile_list = []

rep_dict = {"level 1":[],
            "level 4":[],
            "level 5":[]}
duration_dict = {"level 1":[],
                "level 4":[],
                "level 5":[]}
clear_dict = {"level 1":[],
              "level 4":[],
              "level 5":[]}
kill_dict = {"level 1":[],
             "level 4":[],
             "level 5":[]}
healthloss_dict = {"level 1":[],
                   "level 4":[],
                   "level 5":[]}



for events_file in eventsfile_list:
    print(events_file)
    # Get general info
    sub = events_file.split('/')[7]
    ses = events_file.split('/')[8]
    run = events_file.split('/')[-1].split('_')[3].split('-')[1][-1]

    # Check if fmrifile exists
    fmri_file = op.join(DATA_PATH, "shinobi.fmriprep", sub, ses, "func", f"{sub}_{ses}_task-shinobi_run-{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
    fmrifile_ok = op.isfile(fmri_file)
    if fmrifile_ok:
        nvol = nib.load(fmri_file).shape[-1]
        events = pd.read_csv(events_file, sep="\t")
        reps_df = events[events["trial_type"]=="gym-retro_game"]
        ntotreps = len(reps_df)
        # Init 
        usable_reps = 0
        nframes = {"level 1":0,
                   "level 4":0,
                   "level 5":0}
        clear = {"level 1":0,
                 "level 4":0,
                 "level 5":0}
        repetitions = {"level 1":0,
                       "level 4":0,
                       "level 5":0}
        kills = {"level 1":0,
                 "level 4":0,
                 "level 5":0}
        healthloss = {"level 1":0,
                      "level 4":0,
                      "level 5":0}
        
        for _, rep in reps_df.iterrows():
            if type(rep["stim_file"]) == str and not "Missing file" in rep["stim_file"]:
                bk2_path = op.join(DATA_PATH, "shinobi", rep["stim_file"])
                frame_count = count_frames(bk2_path)
                usable_reps += 1
                if rep["level"] == "level-1":
                    nlvl1 += 1
                    nframes_lvl1 += frame_count
                elif rep["level"] == "level-4":
                    nlvl4 += 1
                    nframes_lvl4 += frame_count
                elif rep["level"] == "level-5":
                    nlvl5 += 1
                    nframes_lvl5 += frame_count
                    
                json_fname = rep["stim_file"].replace(".bk2", ".json")
                with open(op.join(DATA_PATH, "shinobi", json_fname)) as f:
                    sidecar = json.load(f)
                nclear += int(sidecar["cleared"])
                nhealthloss += int(sidecar["total health lost"])
                nkill += int(sidecar["enemies killed"])

        sub_list.append(sub)
        ses_list.append(ses)
        run_list.append(f"run-0{run}")
        fmrifile_list.append(fmri_file)
        nvol_list.append(nvol)
        nrep_total_list.append(ntotreps)
        nrep_usable_list.append(usable_reps)
        nlvl1_list.append(nlvl1)
        nlvl4_list.append(nlvl4)
        nlvl5_list.append(nlvl5)
        nframes_lvl1_list.append(nframes_lvl1)
        nframes_lvl4_list.append(nframes_lvl4)
        nframes_lvl5_list.append(nframes_lvl5)
        
        nclear_list.append(nclear)
        nhealthloss_list.append(nhealthloss)
        nkill_list.append(nkill)



data_df = pd.DataFrame({
    "sub" : sub_list,
    "ses" : ses_list,
    "run" : run_list,
    "fmri_file" : fmrifile_list,
    "nvol" : nvol_list,
    "nrep_total" : nrep_total_list,
    "nrep_usable" : nrep_usable_list,
    "nlvl1" : nlvl1_list,
    "nlvl4" : nlvl4_list,
    "nlvl5" : nlvl5_list,
    "nclear" : nclear_list,
    "nhealthloss" : nhealthloss_list,
    "nkill" : nkill_list,

})
data_df.to_csv(dataset_fname, index=False)
    #return data_df


dataset_fname = op.join(DATA_PATH, "processed", "shinobi_dataset_description_test.csv")

#if not op.isfile(dataset_fname):
#    data_df = build_dataset_description(dataset_fname)
#else:
#    data_df = pd.read_csv(dataset_fname)